# Building a data analysis company with Group Chat

In [1]:
## import required libraries
import os
from autogen import UserProxyAgent, AssistantAgent, GroupChat, GroupChatManager
from autogen import ConversableAgent
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
## llm configuration
llm_config = {
    "model": "gpt-4o",
    "api_key": os.getenv("OPENAI_API_KEY")
}

## The task!

In [3]:
task = "Analyze the daily closing prices of Apple (AAPL) stock for the past 1 month and create a brief reporst."

## Create the Agents

This group chat will include these agents:

- User_proxy or Admin: to allow the user to comment on the report and ask the writer to refine it.
- Planner: to determine relevant information needed to complete the task.
- Engineer: to write code using the defined plan by the planner.
- Executor: to execute the code written by the engineer.
- Writer: to write the report.

In [15]:
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="Give the task, and send instructions to the writer to refine the stock analysis report.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS"
)

In [16]:
## planner agent
planner = ConversableAgent(
    name="Planner",
    system_message="""
    Given a task, please determine what stock information is needed to complete the task. 
    Please note that the information will all be retrieved using Python code. 
    Please only suggest information that can be retrieved using Python code. 
    After each step is done by others, check the progress and instruct the remaining steps. 
    If a step fails, try to workaround.
    """,
    llm_config=llm_config,
)

In [17]:
## engineer agent
engineer = AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code to fetch and analyze stock data based on the plan provided by the planner.",
)

In [18]:
## executor agent
executor = ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the engineer and report the stock data results.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 6,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [19]:
## writer agent
writer = ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Stock Analysis Report Writer. Please write stock analysis reports in markdown format (with relevant titles) and put the content in pseudo ```md``` code block. You take feedback from the admin and refine your report.",
    description="Writer. Write stock analysis reports based on the code execution results and take feedback from the admin to refine the report."
)

## Define the group chat

In [20]:
groupchat = GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

In [21]:
manager = GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

## Start the group chat

In [22]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task
)

Admin (to chat_manager):

Analyze the daily closing prices of Apple (AAPL) stock for the past 1 month and create a brief reporst.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To complete this task, we need the following stock information for Apple (AAPL):

1. **Daily Closing Prices:** Retrieve daily closing prices of AAPL for the past month.
2. **Dates:** Dates corresponding to each closing price, to arrange the data chronologically.

Here are the step-by-step instructions to gather the necessary data:

### Step 1: Retrieve Stock Data
- Use a reliable financial data library in Python, such as `yfinance`.
- Import the library and download the historical stock data for AAPL for the past month.

Python code snippet to download data:

```python
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Define the ticker symbol for Apple
ticker = 


--------------------------------------------------------------------------------

Next speaker: Executor


>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
Executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 
Stock Analysis Report for AAPL:
Average Closing Price: $211.55
Highest Closing Price: $229.35
Lowest Closing Price: $202.38
Figure(1000x500)


--------------------------------------------------------------------------------

Next speaker: Writer


>>>>>>>> USING AUTO REPLY...
Writer (to chat_manager):

Great! The script executed successfully, and it looks like we have some interesting insights about Apple (AAPL) stock. Here's a concise analysis based on the output:

```md
# AAPL Stock Analysis Report - Last 1 Month

## Summary Statistics
- **Average Closing Price:** $211.55
- **Highest Closing Price:** $229.35
- **Lowest Closing Price:** $202.38

## Daily Price Change
The daily price changes were calculated and added to further analyze the 